# IBM Applied Data Science Capstone Course by Coursera
## Week 5 Final Report
### Opening a New Restaurant in Shanghai, China

* Build a dataframe of administrative divisions of Shanghai by web scraping the data from Wikipedia  
* Receive the geographical coordinates of the list   
* Obtain the venue data for the neighborhoods from Foursquare API  
* Explore and cluster the neighborhoods  
* Select the best cluster to open a new restaurant

### 1. Import Liraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


In [ ]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


### 2. Scrap Data from Wikipedia page into a DataFrame

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_administrative_divisions_of_Shanghai'
dfs = pd.read_html(url)

,Municipality of Shanghai,Municipality of Shanghai.1
0,Municipality,Municipality
1,Seat: Huangpu District,Seat: Huangpu District
2,County level divisions,County level divisions
3,Sub-provincial new areas,1
4,Districts,15
5,Township level divisions,Township level divisions
6,Towns,106
7,Townships,2
8,Subdistricts,106
9,Villages level divisions,Villages level divisions


In [18]:
df = dfs[3]
df

Unnamed: 0_level_0                    County Level                        \
   Unnamed: 0_level_1                            Name Chinese  Hanyu Pinyin   
0                 NaN  Huangpu District[4](City seat)     黄浦区    Huángpǔ Qū   
1                 NaN                  Xuhui District     徐汇区      Xúhuì Qū   
2                 NaN              Changning District     长宁区  Chángníng Qū   
3                 NaN                Jing'an District     静安区    Jìng'ān Qū   
4                 NaN                  Putuo District     普陀区      Pǔtuó Qū   
5                 NaN                Hongkou District     虹口区    Hóngkǒu Qū   
6                 NaN                 Yangpu District     杨浦区     Yángpǔ Qū   
7                 NaN                 Pudong New Area    浦东新区  Pǔdōng Xīnqū   
8                 NaN                Minhang District     闵行区    Mǐnháng Qū   
9                 NaN                Baoshan District     宝山区    Bǎoshān Qū   
10                NaN                Jiading District     嘉定区    Jiādìng Qū   
11                NaN                Jinshan District     金山区    Jīnshān Qū   
12                NaN              Songjiang District     松江区  Sōngjiāng Qū   
13                NaN                 Qingpu District     青浦区     Qīngpǔ Qū   
14                NaN               Fengxian District     奉贤区   Fèngxián Qū   
15                NaN              Chongming District     崇明区  Chóngmíng Qū   

                                                      \
   Division code[2] Division code[2].1 Area (km²)[3]   
0            310101                HGP         20.46   
1            310104                XHI         54.76   
2            310105                CNQ         38.30   
3            310106                JAQ         36.88   
4            310107                PTQ         54.83   
5            310109                HKQ         23.48   
6            310110                YPU         60.73   
7            310115                PDX       1210.41   
8            310112                MHQ        370.75   
9            310113                BAO        270.99   
10           310114                JDG        464.20   
11           310116                JSH        586.05   
12           310117                SOJ        605.64   
13           310118                QPU        670.14   
14           310120                FXI        687.39   
15           310151                CMG       1185.49   

                                               
   Population (2018 census)[3] Density (/km²)  
0                       653800          31955  
1                      1084400          19803  
2                       694000          18120  
3                      1062800          28818  
4                      1281900          23380  
5                       797000          33944  
6                      1312700          21615  
7                      5550200           4585  
8                      2543500           6860  
9                      2042300           7536  
10                     1588900           3423  
11                      805000           1374  
12                     1762200           2910  
13                     1219100           1819  
14                     1152000           1676  
15                      688100            580

In [23]:
df_new = df.drop(columns=['Unnamed: 0_level_0'])
df_new

/Users/jeffreyhu/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


County Level                                         \
                              Name Chinese  Hanyu Pinyin Division code[2]   
0   Huangpu District[4](City seat)     黄浦区    Huángpǔ Qū           310101   
1                   Xuhui District     徐汇区      Xúhuì Qū           310104   
2               Changning District     长宁区  Chángníng Qū           310105   
3                 Jing'an District     静安区    Jìng'ān Qū           310106   
4                   Putuo District     普陀区      Pǔtuó Qū           310107   
5                 Hongkou District     虹口区    Hóngkǒu Qū           310109   
6                  Yangpu District     杨浦区     Yángpǔ Qū           310110   
7                  Pudong New Area    浦东新区  Pǔdōng Xīnqū           310115   
8                 Minhang District     闵行区    Mǐnháng Qū           310112   
9                 Baoshan District     宝山区    Bǎoshān Qū           310113   
10                Jiading District     嘉定区    Jiādìng Qū           310114   
11                Jinshan District     金山区    Jīnshān Qū           310116   
12              Songjiang District     松江区  Sōngjiāng Qū           310117   
13                 Qingpu District     青浦区     Qīngpǔ Qū           310118   
14               Fengxian District     奉贤区   Fèngxián Qū           310120   
15              Chongming District     崇明区  Chóngmíng Qū           310151   

                                                                                
   Division code[2].1 Area (km²)[3] Population (2018 census)[3] Density (/km²)  
0                 HGP         20.46                      653800          31955  
1                 XHI         54.76                     1084400          19803  
2                 CNQ         38.30                      694000          18120  
3                 JAQ         36.88                     1062800          28818  
4                 PTQ         54.83                     1281900          23380  
5                 HKQ         23.48                      797000          33944  
6                 YPU         60.73                     1312700          21615  
7                 PDX       1210.41                     5550200           4585  
8                 MHQ        370.75                     2543500           6860  
9                 BAO        270.99                     2042300           7536  
10                JDG        464.20                     1588900           3423  
11                JSH        586.05                      805000           1374  
12                SOJ        605.64                     1762200           2910  
13                QPU        670.14                     1219100           1819  
14                FXI        687.39                     1152000           1676  
15                CMG       1185.49                      688100            580

In [24]:
df_new.columns = df_new.columns.droplevel()
df_new

,Name,Chinese,Hanyu Pinyin,Division code[2],Division code[2].1,Area (km²)[3],Population (2018 census)[3],Density (/km²)
0,Huangpu District[4](City seat),黄浦区,Huángpǔ Qū,310101,HGP,20.46,653800,31955
1,Xuhui District,徐汇区,Xúhuì Qū,310104,XHI,54.76,1084400,19803
2,Changning District,长宁区,Chángníng Qū,310105,CNQ,38.30,694000,18120
3,Jing'an District,静安区,Jìng'ān Qū,310106,JAQ,36.88,1062800,28818
4,Putuo District,普陀区,Pǔtuó Qū,310107,PTQ,54.83,1281900,23380
5,Hongkou District,虹口区,Hóngkǒu Qū,310109,HKQ,23.48,797000,33944
6,Yangpu District,杨浦区,Yángpǔ Qū,310110,YPU,60.73,1312700,21615
7,Pudong New Area,浦东新区,Pǔdōng Xīnqū,310115,PDX,1210.41,5550200,4585
8,Minhang District,闵行区,Mǐnháng Qū,310112,MHQ,370.75,2543500,6860
9,Baoshan District,宝山区,Bǎoshān Qū,310113,BAO,270.99,2042300,7536


In [25]:
df_result = df_new.rename(columns={"Population (2018 census)[3]": "Population", "Area (km²)[3]": "Area (km²)"})
df_result

,Name,Chinese,Hanyu Pinyin,Division code[2],Division code[2].1,Area (km²),Population,Density (/km²)
0,Huangpu District[4](City seat),黄浦区,Huángpǔ Qū,310101,HGP,20.46,653800,31955
1,Xuhui District,徐汇区,Xúhuì Qū,310104,XHI,54.76,1084400,19803
2,Changning District,长宁区,Chángníng Qū,310105,CNQ,38.30,694000,18120
3,Jing'an District,静安区,Jìng'ān Qū,310106,JAQ,36.88,1062800,28818
4,Putuo District,普陀区,Pǔtuó Qū,310107,PTQ,54.83,1281900,23380
5,Hongkou District,虹口区,Hóngkǒu Qū,310109,HKQ,23.48,797000,33944
6,Yangpu District,杨浦区,Yángpǔ Qū,310110,YPU,60.73,1312700,21615
7,Pudong New Area,浦东新区,Pǔdōng Xīnqū,310115,PDX,1210.41,5550200,4585
8,Minhang District,闵行区,Mǐnháng Qū,310112,MHQ,370.75,2543500,6860
9,Baoshan District,宝山区,Bǎoshān Qū,310113,BAO,270.99,2042300,7536


In [27]:
df_result['Name'][0] = 'Huangpu District'
df_result

/Users/jeffreyhu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Name,Chinese,Hanyu Pinyin,Division code[2],Division code[2].1,Area (km²),Population,Density (/km²)
0,Huangpu District,黄浦区,Huángpǔ Qū,310101,HGP,20.46,653800,31955
1,Xuhui District,徐汇区,Xúhuì Qū,310104,XHI,54.76,1084400,19803
2,Changning District,长宁区,Chángníng Qū,310105,CNQ,38.30,694000,18120
3,Jing'an District,静安区,Jìng'ān Qū,310106,JAQ,36.88,1062800,28818
4,Putuo District,普陀区,Pǔtuó Qū,310107,PTQ,54.83,1281900,23380
5,Hongkou District,虹口区,Hóngkǒu Qū,310109,HKQ,23.48,797000,33944
6,Yangpu District,杨浦区,Yángpǔ Qū,310110,YPU,60.73,1312700,21615
7,Pudong New Area,浦东新区,Pǔdōng Xīnqū,310115,PDX,1210.41,5550200,4585
8,Minhang District,闵行区,Mǐnháng Qū,310112,MHQ,370.75,2543500,6860
9,Baoshan District,宝山区,Bǎoshān Qū,310113,BAO,270.99,2042300,7536


In [28]:
df_result = df_result[['Name','Area (km²)', 'Population', 'Density (/km²)']]
df_result

,Name,Area (km²),Population,Density (/km²)
0,Huangpu District,20.46,653800,31955
1,Xuhui District,54.76,1084400,19803
2,Changning District,38.30,694000,18120
3,Jing'an District,36.88,1062800,28818
4,Putuo District,54.83,1281900,23380
5,Hongkou District,23.48,797000,33944
6,Yangpu District,60.73,1312700,21615
7,Pudong New Area,1210.41,5550200,4585
8,Minhang District,370.75,2543500,6860
9,Baoshan District,270.99,2042300,7536


### 3. Get the geographical coordinates

In [32]:
import geocoder # to get coordinates

# define a function to get coordinates
def get_latlng(name):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Shanghai, China'.format(name))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [33]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(name) for name in df_result["Name"].tolist() ]
coords

[[31.23780000000005, 121.47810000000004],
 [31.195940000000064, 121.44709000000012],
 [31.217390000000023, 121.42105000000004],
 [31.220000000000027, 121.41583000000003],
 [31.251000000000033, 121.38970000000006],
 [31.250000000000057, 121.48917000000006],
 [31.261930000000064, 121.51904000000002],
 [31.235130000000026, 121.52759000000003],
 [31.10880000000003, 121.37472000000002],
 [31.416390000000035, 121.48000000000002],
 [31.366370000000074, 121.22153000000003],
 [30.920250000000067, 121.25199000000009],
 [31.03595000000007, 121.21460000000002],
 [31.153940000000034, 121.11408000000006],
 [30.833810000000028, 121.52128000000005],
 [31.389950000000056, 121.68385]]

In [34]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
df_result['Latitude'] = df_coords['Latitude']
df_result['Longitude'] = df_coords['Longitude']

df_result

/Users/jeffreyhu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/jeffreyhu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Name,Area (km²),Population,Density (/km²),Latitude,Longitude
0,Huangpu District,20.46,653800,31955,31.23780,121.47810
1,Xuhui District,54.76,1084400,19803,31.19594,121.44709
2,Changning District,38.30,694000,18120,31.21739,121.42105
3,Jing'an District,36.88,1062800,28818,31.22000,121.41583
4,Putuo District,54.83,1281900,23380,31.25100,121.38970
5,Hongkou District,23.48,797000,33944,31.25000,121.48917
6,Yangpu District,60.73,1312700,21615,31.26193,121.51904
7,Pudong New Area,1210.41,5550200,4585,31.23513,121.52759
8,Minhang District,370.75,2543500,6860,31.10880,121.37472
9,Baoshan District,270.99,2042300,7536,31.41639,121.48000


In [35]:
# save the DataFrame as CSV file
df_result.to_csv("df_result.csv")

### 4. Create a map of Shanghai with administrative_divisions superimposed on top

In [37]:
# get the coordinates of Kuala Lumpur
address = 'Shanghai, China'

geolocator = Nominatim(user_agent="user_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Shanghai, China {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Shanghai, China 31.2252985, 121.4890497.


In [38]:
# create map of Toronto using latitude and longitude values
map_sh = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, name in zip(df_result['Latitude'], df_result['Longitude'], df_result['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sh)  
    
map_sh

In [39]:
# save the map as HTML file
map_sh.save('map_sh.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [40]:
# define Foursquare Credentials and Version
CLIENT_ID = 'NDTR1BTPHIMBVIDLYUYBFLQSNYVZBJIOC5BWX0NSCV0WU0OO' # your Foursquare ID
CLIENT_SECRET = 'MSYYHG34P13UEKPYHVI4U1XHISC5VCSXVZXZDKMGRVX4HVFA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NDTR1BTPHIMBVIDLYUYBFLQSNYVZBJIOC5BWX0NSCV0WU0OO
CLIENT_SECRET:MSYYHG34P13UEKPYHVI4U1XHISC5VCSXVZXZDKMGRVX4HVFA


## Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [41]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, name in zip(df_result['Latitude'], df_result['Longitude'], df_result['Name']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            name,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [42]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Name', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(712, 7)


,Name,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Huangpu District,31.2378,121.4781,Bund Plaza,31.239211,121.479741,Department Store
1,Huangpu District,31.2378,121.4781,Grand Central Hotel Shanghai (上海大酒店),31.237379,121.476754,Hotel
2,Huangpu District,31.2378,121.4781,M1NT Restaurant & Grill,31.236920,121.479641,Restaurant
3,Huangpu District,31.2378,121.4781,Épices & Foie-gras,31.237557,121.479580,French Restaurant
4,Huangpu District,31.2378,121.4781,The Shanghai EDITION (上海爱迪逊酒店),31.240001,121.481678,Hotel


In [43]:
venues_df.groupby(["Name"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Name,,,,,,
Baoshan District,5,5,5,5,5,5
Changning District,100,100,100,100,100,100
Chongming District,4,4,4,4,4,4
Fengxian District,4,4,4,4,4,4
Hongkou District,100,100,100,100,100,100
Huangpu District,100,100,100,100,100,100
Jiading District,6,6,6,6,6,6
Jing'an District,100,100,100,100,100,100
Minhang District,27,27,27,27,27,27


In [44]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 130 uniques categories.


In [45]:
venues_df['VenueCategory'].unique()[:50]

array(['Department Store', 'Hotel', 'Restaurant', 'French Restaurant',
       'Pedestrian Plaza', 'Cantonese Restaurant', 'Lounge', 'Waterfront',
       'Café', 'Nightclub', 'Japanese Restaurant', 'Italian Restaurant',
       'Yunnan Restaurant', 'Hotpot Restaurant', 'Dumpling Restaurant',
       'Jazz Club', 'Bar', 'Shanghai Restaurant', 'Shopping Mall',
       'BBQ Joint', 'Chinese Restaurant', 'Roof Deck', 'Steakhouse',
       'Karaoke Bar', 'Coffee Shop', 'American Restaurant', 'Candy Store',
       'Cocktail Bar', 'Hostel', 'Theater', 'Plaza',
       'Vegetarian / Vegan Restaurant', 'Concert Hall',
       'Sculpture Garden', 'Art Museum', 'Spanish Restaurant', 'Park',
       'Hotel Bar', 'Mexican Restaurant', 'Gym', 'Fast Food Restaurant',
       'Spa', 'Grocery Store', 'Beer Bar', 'Garden', 'Massage Studio',
       'Scenic Lookout', 'Yoga Studio', 'Peruvian Restaurant',
       'Xinjiang Restaurant'], dtype=object)

### 6. Analyze Each Neighborhood

In [46]:
# one hot encoding
sh_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sh_onehot['Name'] = venues_df['Name'] 

# move neighborhood column to the first column
fixed_columns = [sh_onehot.columns[-1]] + list(sh_onehot.columns[:-1])
sh_onehot = sh_onehot[fixed_columns]

print(sh_onehot.shape)
sh_onehot.head()

(712, 131)


,Name,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Big Box Store,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Creperie,Deli / Bodega,Department Store,Dongbei Restaurant,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food & Drink Shop,French Restaurant,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hockey Arena,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pie Shop,Pier,Pizza Place,Plaza,Pub,Public Art,Racetrack,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Theater,Toll Booth,Toll Plaza,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant
0,Huangpu District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Huangpu District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Huangpu District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Huangpu District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Huangpu District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
sh_grouped = sh_onehot.groupby(["Name"]).mean().reset_index()

print(sh_grouped.shape)
sh_grouped

(15, 131)


,Name,American Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bakery,Bar,Beach,Beer Bar,Big Box Store,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Buffet,Burger Joint,Bus Station,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Creperie,Deli / Bodega,Department Store,Dongbei Restaurant,Dumpling Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food & Drink Shop,French Restaurant,Furniture / Home Store,Garden,General Entertainment,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,History Museum,Hockey Arena,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Indian Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pie Shop,Pier,Pizza Place,Plaza,Pub,Public Art,Racetrack,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Theater,Toll Booth,Toll Plaza,Turkish Restaurant,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Xinjiang Restaurant,Yoga Studio,Yunnan Restaurant,Zhejiang Restaurant
0,Baoshan District,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.20,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.200000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000,0.00,0.00,0.400000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.200000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000
1,Changning District,0.000000,0.00,0.01,0.00,0.010000,0.010000,0.050000,0.020000,0.00,0.03,0.000000,0.01,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.00,0.01,0.00,0.000,0.080000,0.00,0.010000,0.02,0.050000,0.00,0.00,0.00,0.01,0.010000,0.00,0.00,0.00,0.02,0.010000,0.000000,0.000000,0.010000,0.04,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.01,0.000,0.000000,0.010000,0.050000,0.000,0.010000,0.00,0.010000,0.040000,0.01,0.05,0.01,0.00,0.00,0.010000,0.030000,0.000,0.01,0.010000,0.00,0.000000,0.00,0.00,0.01,0.00,0.000000,0.010000,0.01,0.01,0.010,0.00,0.02,0.010000,0.00,0.020000,0.00,0.00,0.01,0.00,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.010000,0.02,0.010000,0.01,0.000000,0.01,0.00,0.01,0.000000,0.000000,0.00,0.010000,0.000000,0.020000,0.00,0.01,0.00,0.00,0.000000,0.000000,0.000000,0.01,0.00,0.02,0.000000,0.01,0.02,0.000000
2,Chongming District,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.250000,0.00,0.00,0.000000,0.00,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000,0.250000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0

#### Let's print each neighborhood along with the top 5 most common venues

In [48]:
num_top_venues = 5

for hood in sh_grouped['Name']:
    print("----"+hood+"----")
    temp = sh_grouped[sh_grouped['Name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baoshan District----
                  venue  freq
0                  Park   0.4
1  Fast Food Restaurant   0.2
2         Boat or Ferry   0.2
3         Shopping Mall   0.2
4   American Restaurant   0.0


----Changning District----
                 venue  freq
0   Chinese Restaurant  0.08
1                Hotel  0.05
2               Bakery  0.05
3          Coffee Shop  0.05
4  Japanese Restaurant  0.05


----Chongming District----
                 venue  freq
0                Hotel  0.25
1   Chinese Restaurant  0.25
2                  Bar  0.25
3        Boat or Ferry  0.25
4  American Restaurant  0.00


----Fengxian District----
               venue  freq
0  Korean Restaurant  0.25
1              Hotel  0.25
2              Beach  0.25
3               Café  0.25
4               Pier  0.00


----Hongkou District----
                venue  freq
0               Hotel  0.18
1              Lounge  0.07
2          Restaurant  0.06
3           Hotel Bar  0.05
4  Italian Restaurant  0.05


--

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Name'] = sh_grouped['Name']

for ind in np.arange(sh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baoshan District,Park,Shopping Mall,Boat or Ferry,Fast Food Restaurant,Zhejiang Restaurant,Garden,Furniture / Home Store,French Restaurant,Food & Drink Shop,Flower Shop
1,Changning District,Chinese Restaurant,Coffee Shop,Japanese Restaurant,Bakery,Hotel,Italian Restaurant,French Restaurant,Lounge,Beer Bar,Café
2,Chongming District,Chinese Restaurant,Boat or Ferry,Hotel,Bar,Zhejiang Restaurant,General Entertainment,Garden,Furniture / Home Store,French Restaurant,Food & Drink Shop
3,Fengxian District,Korean Restaurant,Beach,Hotel,Café,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Gourmet Shop,Electronics Store
4,Hongkou District,Hotel,Lounge,Restaurant,Italian Restaurant,Hotel Bar,French Restaurant,Chinese Restaurant,Shopping Mall,Dumpling Restaurant,Department Store


In [51]:
neighborhoods_venues_sorted

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baoshan District,Park,Shopping Mall,Boat or Ferry,Fast Food Restaurant,Zhejiang Restaurant,Garden,Furniture / Home Store,French Restaurant,Food & Drink Shop,Flower Shop
1,Changning District,Chinese Restaurant,Coffee Shop,Japanese Restaurant,Bakery,Hotel,Italian Restaurant,French Restaurant,Lounge,Beer Bar,Café
2,Chongming District,Chinese Restaurant,Boat or Ferry,Hotel,Bar,Zhejiang Restaurant,General Entertainment,Garden,Furniture / Home Store,French Restaurant,Food & Drink Shop
3,Fengxian District,Korean Restaurant,Beach,Hotel,Café,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Gourmet Shop,Electronics Store
4,Hongkou District,Hotel,Lounge,Restaurant,Italian Restaurant,Hotel Bar,French Restaurant,Chinese Restaurant,Shopping Mall,Dumpling Restaurant,Department Store
5,Huangpu District,Hotel,Lounge,French Restaurant,Coffee Shop,Café,Italian Restaurant,Chinese Restaurant,Shopping Mall,Park,Shanghai Restaurant
6,Jiading District,Park,Asian Restaurant,Toll Plaza,Bus Station,Metro Station,Hotel,Zhejiang Restaurant,Food & Drink Shop,General Entertainment,Garden
7,Jing'an District,Bakery,Chinese Restaurant,Coffee Shop,Hotel,Japanese Restaurant,Shopping Mall,Café,French Restaurant,Lounge,Beer Bar
8,Minhang District,Coffee Shop,Fast Food Restaurant,Shopping Mall,Bakery,Café,Metro Station,Zhejiang Restaurant,Supermarket,Clothing Store,Park
9,Pudong New Area,Coffee Shop,Hotel,Shopping Mall,Fast Food Restaurant,Chinese Restaurant,Pizza Place,Sandwich Place,Café,Sushi Restaurant,Breakfast Spot


### 7. Observations and Recommendations

For the question that we want to answer, "how to open a restaurant in Shanghai" depends on whick kind of restaurant you want to choose from.

Base on the principle of __knock-on effect__, we advice you'd better open a restaurant which is located near other similar ones.

For this reason, we get some conclusions as follows: 

1. Qingpu District will be a best choice if you want to open a __coffer shop__ because its frequency is as high as 0.50; Minhang, Yangpu and Pudong New area can also be considered since their frequency is more than 0.20
2. If you want to open a Chinese restaurant, then Changning and Chongming District should be your first consideration since it is the 1st most common venue in this area. 
3. Fengxian District is a good choice for Korean style and Putuo District is suitable for Fast Food restuarant.
4. For westerners, Huangpu and Xuhui district can be considered first because they have the highest rate of frequency for western style restaurant,say, French, Italian, Cocktail Bar, etc. 